# Importing python packages

In [ ]:
import glob
from multiprocessing import Pool
import time
from tqdm import tqdm
import pandas as pd

# Importing SSPARQ setup

In [ ]:
from parameters_py.config import (
					WAVEFORM_DIR,CATALOG_FILE,XML_DIR,SSPARQ_OUTPUT,num_processes
				   )

# Importing SSPARQ functions

In [ ]:
from src.plot import (
					plotting_event_orientation
				   )

# Plot a comprehensive overview of a seismic event and station orientation analysis.

### Reading station

In [ ]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))
STATIONS_xml = STATIONS_xml

STATIONS_lst = [i.split('/')[-1].split('.xml')[0] for i in STATIONS_xml]

print('Number of stations:',len(STATIONS_lst))

### Running multiprocessing function given a input list

In [ ]:
start_time = time.time()

for sta in STATIONS_lst:
    print('Station: ',sta)
    FEATHER_FILES_LST = [pd.read_feather(i) for i in glob.glob(SSPARQ_OUTPUT+'FEATHER_FILES/METRICS/*'+sta+'/*')]
    station_df = pd.concat(FEATHER_FILES_LST)

    with Pool(processes=20) as p:
        max_ = len(station_df)
        with tqdm(total=max_) as pbar:
            for result in p.imap_unordered(plotting_event_orientation,station_df.iterrows()):
                pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')